In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.4` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:0.10.9` 

In [ ]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


In [ ]:
import org.apache.spark.sql._

In [ ]:
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .config("spark.testing.memory", 471859200)
    .config("spark.executor.instances", "4")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
}

import spark.implicits._

In [ ]:
var df = spark.read.format("parquet").load("data/train")

In [ ]:
//  посмотрим на список колонок
/*
for (c <- df.columns) {
    println(c)
}
*/

In [ ]:
df.createOrReplaceTempView("trainDF")
var df_exp = spark.sql("""
    select t.*, explode(feedback) as fb_exp from trainDF t 
""")
df_exp.createOrReplaceTempView("trainDF_exp")

# Задание 1 - Постройте топ популярных групп на портале

In [ ]:
spark.sql("""
    select metadata_ownerId, count(*) as like_cnt from trainDF_exp
    where fb_exp = 'Liked'
    group by metadata_ownerId
    order by like_cnt desc
""").show(10)

# Задание 2 - Построите гистограммы популярности/активности групп на портале по времени суток

In [ ]:
spark.sql("""
    select distinct fb_exp from trainDF_exp
""").show()

In [ ]:
var activityDF = spark.sql("""
    select created_hour as created_hour_a, count(*) as cnt_a from (
        select 
            from_unixtime(metadata_createdAt, 'HH') as created_hour
            , fb_exp
        from trainDF_exp 
        where fb_exp <> 'Ignored'
    ) 
    group by created_hour
""").as("df_activity")

var popularityDF = spark.sql("""
    select created_hour as created_hour_p, count(*) as cnt_p from (
        select 
            from_unixtime(metadata_createdAt, 'HH') as created_hour
            , fb_exp
        from trainDF_exp 
        where fb_exp = 'Liked'
    ) 
    group by created_hour
""").as("df_popularity")



In [ ]:
var joined_df = activityDF.join(
popularityDF
, $"created_hour_a" === $"created_hour_p"
, "inner")

In [ ]:
joined_df.show(24)
joined_df.createOrReplaceTempView("joined_df")

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._



In [ ]:
var j_res = spark.sql("""
    select created_hour_a, cnt_a, cnt_p
    from joined_df
    order by created_hour_a
""")


var hours = Seq[String]()
var cnt_a = Seq[Int]()
var cnt_p = Seq[Int]()

for (row <- j_res.collect()){
    hours = hours :+ row(0).asInstanceOf[String] + 'h'
    cnt_a = cnt_a :+ row(1).asInstanceOf[Long].toInt
    cnt_p = cnt_p :+ row(2).asInstanceOf[Long].toInt
}

In [ ]:
val trace1 = Bar(
  hours,
  cnt_a,
  "Активность",
  text = hours.map(x => "Час " + x),
  // orientation = Orientation.Horizontal,
  marker = Marker(
    color = Color.RGB(49, 130, 189),
    opacity = 0.7))

val trace2 = Bar(
  hours,
  cnt_p,
  "Популярность",
  text = hours.map(x => "Час " + x),
  // orientation = Orientation.Horizontal,
  marker = Marker(
    color = Color.RGB(204, 0, 0),
    opacity = 0.7))

Seq(trace1, trace2).plot("Данные активности и популярности групп по часам")


In [ ]:
// обнулим переменные
activityDF = null
popularityDF = null
joined_df = null
j_res = null
df = null

##### Посольку груфик не отображается в случае, если открыть ноутбук напрямую из гитхаба, продублирую его в виде картинки:

<img src="activity_popularity.png">

# Задание 3 - Посчитайте корреляцию признаков с целевой переменной

#### посмотрим какие типы вообще есть в нашем наборе данных

In [ ]:
var fieldTypes = Seq[String]()
val df_exp_int = 
for ( field <- df_exp.dtypes) {
    fieldTypes = fieldTypes :+ field._2
}
fieldTypes.distinct

In [ ]:

var numColumns = Seq[String]()
var strColumns = Seq[String]()
var dateColumns = Seq[String]()

for ( field <- df_exp.dtypes ) {
    if(field._2 == "StringType")
        strColumns = strColumns :+ field._1
    else if(field._2 == "IntegerType" || field._2 == "DoubleType" || field._2 == "LongType")
        numColumns = numColumns :+ field._1
    else if(field._2 == "DateType")
        dateColumns = dateColumns :+ field._1
}

#### Поскольку в задании нет четкого определения целевой переменной, будем считать, что ей является признак того, поставили лайк

In [ ]:
// и доавим день недели 
var trainDF_exp = spark.sql("""
    select t.*
    , date_format(date, 'u') as week_day_number
    , case when t.fb_exp = 'Liked' then 1 else 0 end as target
    from trainDF_exp t
""")


In [ ]:
import $ivy.`org.apache.spark::spark-mllib:2.4.4`

#### Напишем функцию, которая будет применять StringIndexer к фрейму и обогощать его новой колонкой

In [ ]:
import org.apache.spark.ml.feature.StringIndexer
def customStringIndexer(df:DataFrame, columnName:String, columnNameIx:String) : DataFrame = {
    val indexer = new StringIndexer()
    .setInputCol(columnName)
    .setOutputCol(columnNameIx) 
    
    println("Indexing " + columnName + "...")
    
    return indexer.fit(df).transform(df)
}

#### Применим функцию ко всем строковым колонкам, кроме той, на основе которой мы делали целевую переменную

In [ ]:
for (strColName <- strColumns) {
    // с membership_status какая-то пролема, пока не было времени разобраться, пока просто исключил её 
    if(strColName != "fb_exp" && strColName != "membership_status"){ 
        trainDF_exp = customStringIndexer(trainDF_exp, strColName, strColName + "_ix")
        numColumns = numColumns :+ strColName + "_ix"
    }
}

In [ ]:
// попробуем выбрать новые колоноки, посмотрим что получилось:
trainDF_exp.select(
    "instanceId_objectType_ix",
    "audit_clientType_ix",
    "audit_experiment_ix",
    "metadata_ownerType_ix",
    "metadata_platform_ix"
).show(10)

### Напишем функцию, которая посчитает нам корреляцию признаков с возможностью получить имена колонок, которые коррелируют с целевой переменной на более чем minToReturn

In [ ]:

def getCorr (df:DataFrame, columnNames:Seq[String], target:String, minToReturn:Double) : Seq[String] = {
    var ret = Seq[String]()
    for (col <- columnNames){
        val corr = trainDF_exp.stat.corr(col, target)  
        println(target + " | " + col + " \t " + corr)  
        if(corr.abs >= minToReturn)
            ret = ret :+ col
    }
    return ret
}


val corrColumns = getCorr(
    trainDF_exp
    ,numColumns
    , "target"
    , 0.05)
 

In [ ]:
// псоморим на список колонок, которые корелируют с целевой переменной более чем на 0.05  
corrColumns

## Подведем итоги

* Больше всего лайков собрали посты групп с идентификаторами 37463, 11222, 18942
* Активность и популяроность не сильно зависит от вреени дня
* Судя по предварительному анализу крайне мало признаков коррелируют с целевой переменной. Но это не означает, что её вовсе нет, она может быть и неявной.

* Категориальные переменные в идеале надо было бы прогнать через One Hot Encoding, быть может лайки зависят от конкретных значений в той или иной категориальной колонки, но для ДЗ прикручивать OHE уже не усспеваю т.к. и так на домашку ушло уйму времени.

## полезные ссылочки на тему:
* конвертируем категориальные признаки в числовые https://spark.apache.org/docs/2.4.4/ml-features.html#stringindexer
* применяем OHE https://spark.apache.org/docs/2.4.4/ml-features.html#onehotencoderestimator
* векторизируем https://spark.apache.org/docs/latest/ml-features.html#vectorassembler
* нармализуем данные, например при помощи StandardScaller https://spark.apache.org/docs/2.4.4/ml-features.html#standardscaler
* считаем корреляцию https://spark.apache.org/docs/2.4.4/ml-statistics.html#correlation


In [ ]:
/*
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.ml.stat.Correlation

import org.apache.spark.ml.linalg.{Matrix, Vectors}

val colName = "metadata_applicationId"
val targetColName = "target"

println("Getting columns " + colName + ", " + targetColName + "...")
val curColDF = trainDF_exp.select(colName, targetColName).na.fill(0.0)

// векторизуем признаки т.к. для нормализации нужен именно вектор
println("Vectorizing " + colName + "...")
val assembler = new VectorAssembler()
  .setInputCols(Array(colName, targetColName))
  .setOutputCol("result")
val assembledDF = assembler.transform(curColDF)

// нормализуем данные
println("Data normalization ...")
val scaler = new MinMaxScaler()
  .setInputCol("result")
  .setOutputCol("result_scaled")

println("Fitting scaler model...")
val scalerModel = scaler.fit(assembledDF)

println("Transforming data...")
val scaledDF = scalerModel.transform(assembledDF)

// Получаем коэффициент корреляции
println("Calculating correlation between " + colName + " and " + targetColName + " columns...")
val Row(coeff1: Matrix) = Correlation.corr(scaledDF, "result_scaled").head
val corrValue = coeff1(0,1)
println(targetColName + " | " + colName + " \t " + corrValue)  
*/